## COMP5318 - Machine Learning and Data Mining 

## Tutorial 11 - Deep Learning

**Semester 2, 2020**

**Objectives:**

* To learn about Keras
* To learn about Auto-encoders and Convolutional Neural Networks
* To use deep learning tricks such as max pooling and dropout

**Instructions:**

* Exercises to be completed on IPython notebook such as: 
   * Ipython 3 (Jupyter) notebook installed on your computer http://jupyter.org/install (you need to have Python installed first https://docs.python.org/3/using/index.html )
   * Web-based Ipython notebooks such as Google Colaboratory https://colab.research.google.com/ 
   
* If you are using Jupyter intalled on your computer, Go to File->Open. Drag and drop "lab11.ipynb" file to the home interface and click upload. 
* If you are using Google Colaboratory, Click File->Upload notebook, and and upload "lab11.ipynb" file
* Complete exercises in "lab11.ipynb".
* To run the cell you can press Ctrl-Enter or hit the Play button at the top.
* Complete all exercises marked with **TODO**.
* Save your file when you are done with the exercises, so you can show your tutor next week.

Lecturers: Nguyen Hoang Tran 

Tutors: Canh Dinh, Chen Chen, Claire Hardgrove, Fengxiang He, Henry Weld, Yixuan Zhang, Zhiyi Wang, Thomas Selvaraj.

## Basic Linear Regression with Keras
Keras is a neural networks API running on top of TensorFlow or Theano, seamlessly running on CPUs and GPUs.

Check out https://keras.io/.

To get familiar with Keras, we first perfor basic linear regression, which is a specific case of a single layer neural netork.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Generate dataset
trX = np.linspace(-1, 1, 101)
# create a y value which is approximately linear but with some random noise
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33

In [ ]:
# Linear regression model
model = Sequential()
# TODO: add a layer using linear activation, dimention of input and out put are 1
model.add(Dense(input_dim = , units = , kernel_initializer='uniform', activation= ))

# TODO: compile model using sgd and meansquare error 
model.compile()

In [ ]:
# Print initial weights
weights = model.layers[0].get_weights()
w_init = weights[0][0][0]
b_init = weights[1][0]
print('Linear regression model is initialized with weight w: %.2f, b: %.2f' % (w_init, b_init))

In [ ]:
# Train
model.fit(trX, trY, epochs=100, verbose=0)

In [ ]:
# Print trained weights
weights = model.layers[0].get_weights()
w = weights[0][0][0]
b = weights[1][0]
print('Linear regression model is trained with weight w: %.2f, b: %.2f' % (w, b))

In [ ]:
plt.plot(trX, trY, label='data')
plt.plot(trX, w_init*trX + b_init, label='init')
plt.plot(trX, w*trX + b, label='prediction')
plt.legend()
plt.show()

In [ ]:
model.summary()

## Loading MNIST data from Keras
We use one of the dataset included in Keras: MNIST (https://en.wikipedia.org/wiki/MNIST_database).

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_val = x_test[:5000,:]
x_test = x_test[5000:,:]
y_val = y_test[:5000]
y_test = y_test[5000:]

## Simple autoencoder
The goal here is to a representation of our data with lower dimension. To do so, autoencoders first transform the data to a low-dimension representation using an encoder network, and then transform the low-dimension representation back to the original space using a decoder network.

Here we start with a single fully-connected neural layer as encoder and as decoder. We will be encoding MNIST digit images (dim=784) into a space of dimension 32, hence the compression factor is 784/32 = 24.5.

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# Reshape data to fit the autoencoder layout and normalize it
x_train_ae = x_train.astype('float32') / 255.
x_test_ae = x_test.astype('float32') / 255.
x_val_ae = x_val.astype('float32') / 255.
x_train_ae = x_train_ae.reshape((len(x_train_ae), np.prod(x_train_ae.shape[1:])))
x_test_ae = x_test_ae.reshape((len(x_test_ae), np.prod(x_test_ae.shape[1:])))
x_val_ae = x_val_ae.reshape((len(x_val_ae), np.prod(x_val_ae.shape[1:])))

Train the autoencoder to reconstruct MNIST digits.

In [ ]:
# Configure the model to use a per-pixel binary crossentropy loss, and Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Train the model for 10 epochs
autoencoder.fit(x_train_ae, x_train_ae, epochs=10, batch_size=256,
                shuffle=True, validation_data=(x_val_ae, x_val_ae))

Vizualize the reconstructed inputs and the encoded representations.

In [ ]:
# Retrieve the encoder and decoder as separate networks
encoder = Model(input_img, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

encoded_imgs = encoder.predict(x_test_ae)
decoded_imgs = decoder.predict(encoded_imgs)

# use Matplotlib
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_ae[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Note that the autoencoder is not fully converged yet as it was trained on only 10 epochs. You may want to try and run it for longer to get better digit reconstruction.

## Convolutional Neural Network

### Simple CNN

Prepare data for CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 1
num_train = 6000 # Max is 60000
num_test = 1000 # Max is 5000
num_val = 1000 # Max is 5000
# Only keep a subset of the data
x_train_cnn = x_train[:num_train,:,:,None]
y_train_cnn = y_train[:num_train]
x_test_cnn = x_test[:num_test,:,:,None]
y_test_cnn = y_test[:num_test]
x_val_cnn = x_test[:num_val,:,:,None]
y_val_cnn = y_test[:num_val]

x_train_cnn = x_train_cnn.astype('float32')
x_test_cnn = x_test_cnn.astype('float32')
x_val_cnn = x_test_cnn.astype('float32')
x_train_cnn /= 255.
x_test_cnn /= 255.
x_val_cnn /= 255.

# Convert class vectors to binary class matrices.
y_train_cnn = keras.utils.to_categorical(y_train_cnn, num_classes)
y_test_cnn = keras.utils.to_categorical(y_test_cnn, num_classes)
y_val_cnn = keras.utils.to_categorical(y_val_cnn, num_classes)

In [ ]:
# Create CNN topology
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train_cnn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_classes))

# TODO: add the last layer as softmax actication 
model.add()

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit model to data
print(epochs)
model.fit(x_train_cnn, y_train_cnn, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val_cnn, y_val_cnn), shuffle=True)

In [ ]:
score = model.evaluate(x_test_cnn, y_test_cnn, batch_size=32, verbose=1)
print("\nTest accuracy is {}%".format(100.0*score[1]))

### Adding dropout and pooling to the CNN
Max pooling and dropouts are tricks that improve deep neural networks.

Max pooling (http://yann.lecun.com/exdb/publis/pdf/boureau-icml-10.pdf) speeds up CNN training and encourages CNN to learn translation-invariant features.
Dropout (https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) is a regularisation technique preventing the model from overfitting the data.

In [ ]:
# Create CNN topology
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train_cnn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#TODO: Change the value of dropout and discuss the differences
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit model to data
epochs = 3
model.fit(x_train_cnn, y_train_cnn, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val_cnn, y_val_cnn), shuffle=True)

Evaluate model score on test set.

In [ ]:
score = model.evaluate(x_test_cnn, y_test_cnn, batch_size=32, verbose=1)
print("\nTest accuracy is {}%".format(100.0*score[1]))